## Load Data

In [ ]:
from google.colab import userdata
username = userdata.get('KAGGLE_USER')
key = userdata.get('KAGGLE_KEY')
# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle competitions download -c liver-ultrasound-detection
!unzip /content/liver-ultrasound-detection.zip && rm -rf /content/liver-ultrasound-detection.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/Hackathon_LV2_ONLINE#2/Medical AI  OCR Hackathon/Liver Lesion/liver-ultrasound-detection.zip' '/content/liver-ultrasound-detection.zip'

In [ ]:
!unzip /content/liver-ultrasound-detection.zip && rm -rf /content/liver-ultrasound-detection.zip

# Visualize Functions

In [ ]:
!pip install matplotlib==3.5

In [ ]:
import gdown

file_id = '1rl1KVbin5cGeFYTfKCiTCBCFJI05g_hM'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'mapping_withPATH.csv'
gdown.download(url, output, quiet=False)

print(f"File downloaded as {output}")


In [ ]:
import pandas as pd

# PATH = 'C:/Users/msari/Research/Project/hackathon_lv2/liver-lesion'
PATH = '/content'

# Load the mapping file
mapping_file_path = f'{PATH}/mapping_withPATH.csv'
# mapping_df = pd.read_excel(mapping_file_path)
mapping_df = pd.read_csv(mapping_file_path)

# Paths to directories
train_image_path = f'{PATH}/train/train/images/'
train_annotation_path = f'{PATH}/train/train/annotations/'
val_image_path = f'{PATH}/val/val/images/'
val_annotation_path = f'{PATH}/val/val/annotations/'
test_image_path = f'{PATH}/test/test/images/'


In [ ]:
mapping_df

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Load the mapping file
# mapping_df = pd.read_excel(f'{PATH}/mapping.xlsx')
# mapping_df = pd.read_csv(f'{PATH}/mapping_withPATH.csv')

# Label mapping
#Label	Description
# 0	FFC
# 1	FFS
# 2	HCC
# 3	cyst
# 4	hemangioma
# 5	dysplastic
# 6	CCA
class_labels = ['FFC', 'FFS', 'HCC', 'cyst', 'hemangioma', 'dysplastic', 'CCA']

def denormalize_bbox(img_shape, bbox):
    img_height, img_width = img_shape[:2]
    x_center, y_center, width, height = bbox
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)
    return x1, y1, x2, y2

def draw_bbox(img, bbox, label):
    x1, y1, x2, y2 = bbox
    if label in ['HCC','CCA']:
      color = (0, 0, 255)  # Red color for the bounding box
    else:
      color = (0, 255, 0)
    thickness = 2
    img = cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    img = cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return img


def visualize_yolo_annotations(image_path, annotation_path, class_labels):
    print(f"Processing image: {image_path}")
    print(f"Annotation file: {annotation_path}")

    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return
    if not os.path.exists(annotation_path):
        print(f"Annotation file not found: {annotation_path}")
        return

    img = cv2.imread(image_path)
    img_with_annotations = img.copy()

    with open(annotation_path, 'r') as file:
        annotations = file.readlines()

    for annotation in annotations:
        parts = annotation.strip().split()
        class_id = int(parts[0])
        bbox = list(map(float, parts[1:]))
        bbox = denormalize_bbox(img.shape, bbox)
        img_with_annotations = draw_bbox(img_with_annotations, bbox, class_labels[class_id])

    # Convert BGR to RGB for displaying with matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_with_annotations_rgb = cv2.cvtColor(img_with_annotations, cv2.COLOR_BGR2RGB)

    # Display original and annotated images side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 7))
    axs[0].imshow(img_rgb)
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(img_with_annotations_rgb)
    axs[1].set_title('Image with Annotations')
    axs[1].axis('off')

    plt.show()

In [ ]:
# Function to print the required paths and sources
def get_paths(source, path):
    filtered_data = data[(data['Source'] == source) & (data['PATH'] == path)]
    data_path = []
    for index, row in filtered_data.iterrows():
      data_path.append([f"train/train/images/{row['Image File']}"])
      # print(f"train/train/images/{row['Image File']}", row['Source'])

    return data_path

# Inputs
data = mapping_df

# Print the results
data_path_train_machine = get_paths('machine', 'Train')
data_path_val_machine = get_paths('machine', 'Val')
data_path_train_mobile = get_paths('mobile', 'Train')
data_path_val_mobile = get_paths('mobile', 'Val')

# Visualize single image

In [ ]:
img_num = 5551

image_path = f'{PATH}/train/train/images/{img_num}.jpg'
annotation_path = f'{PATH}/train/train/annotations/{img_num}.txt'
visualize_yolo_annotations(image_path, annotation_path, class_labels)

# Adding Natural Glare *

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def apply_brightness_contrast(input_img, brightness=0, contrast=0):
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

def add_natural_glare(image, intensity, size):
    h, w = image.shape[:2]
    center_x = np.random.randint(w // 4, 3 * w // 4)
    center_y = np.random.randint(h // 4, 3 * h // 4)
    max_dim = max(h, w)
    mask = np.zeros((h, w), dtype=np.uint8)

    # Create a gradient circle mask
    for i in range(h):
        for j in range(w):
            dist = np.sqrt((i - center_y) ** 2 + (j - center_x) ** 2)
            mask[i, j] = np.clip(255 - int((dist / max_dim) * 255 / size), 0, 255)

    mask = cv2.GaussianBlur(mask, (21, 21), 0)

    # Apply the glare effect using the mask
    glare = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Blend glare to image
    result = cv2.addWeighted(image, 1, glare, intensity, 0)

    return result

## Adding Natural Glare * (Random glare_intensity, glare_size)

In [ ]:
for i in range(10):
  # Load the image
  image1 = cv2.imread(image_path)

  # Apply random brightness and contrast adjustment
  brightness = np.random.randint(0, 50)  # Random brightness between -50 and 50
  contrast = np.random.randint(0, 50)    # Random contrast between -50 and 50
  adjusted_image1 = apply_brightness_contrast(image1, brightness, contrast)
  adjusted_image1 = apply_brightness_contrast(image1, brightness, contrast)

  # Add natural glare effect with random intensity and size
  glare_intensity = np.random.uniform(0.2, 0.5)  # Random glare intensity between 0.2 and 0.5
  glare_size = np.random.uniform(0.3, 0.6)       # Random glare size between 0.3 and 0.6
  adjusted_image1_with_glare = add_natural_glare(adjusted_image1, glare_intensity, glare_size)

  # Show the images
  plt.figure(figsize=(12, 6))

  plt.subplot(1, 2, 1)
  plt.title('Original Image 1')
  plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))

  plt.subplot(1, 2, 2)
  plt.title('Adjusted Image 1 with Natural Glare')
  plt.imshow(cv2.cvtColor(adjusted_image1_with_glare, cv2.COLOR_BGR2RGB))

  plt.show()

# Adding Moire Attack

## latticegen patterns

In [ ]:
!pip install latticegen

In [ ]:
import latticegen

r_k = 0.05
lattice1 = latticegen.hexlattice_gen(r_k, 0, order=1)
lattice1 = np.clip(lattice1 / lattice1.max(), 0, 1).compute()
lattice2 = latticegen.hexlattice_gen(r_k, 5, order=1)
lattice2 = np.clip(lattice2 / lattice2.max(), 0, 1).compute()

fig,axs = plt.subplots(ncols=3, figsize=[10,4])
for i in [0, 1]:
    axs[i].imshow(-lattice1.T, cmap='PiYG',
                      vmax=1,
                      vmin=-1,
                       alpha=lattice1.T
                     )
    axs[i + 1].imshow(lattice2.T, cmap='PiYG',
                   vmax=1,
                   vmin=-1,
                   alpha=lattice2.T)

In [ ]:
r_hBN = r_k * (0.246 / 0.2504)
sublattice_a = latticegen.trilattice_gen(r_hBN, 0, order=1, normalize=True)
sublattice_a = sublattice_a.compute()
# Now add the second shifted sublattice lattice to get a hexagonal lattice
ks = latticegen.generate_ks(r_hBN, 0, sym=6)
x = np.array([ks[1], -ks[2]])
shift = (np.linalg.inv(x / r_hBN).T/(3*r_k)).sum(axis=0).T  # Don't ask, this works
sublattice_b = latticegen.trilattice_gen(r_hBN, 0, order=1,
                                         shift=shift, normalize=True)
sublattice_b = sublattice_b.compute()

fig, axs = plt.subplots(ncols=3, figsize=[10,4])
axs[0].set_title('Sublattice a')
axs[1].set_title('Both sublattices')
axs[2].set_title('Sublattice b')
for i in [0, 1]:
    axs[i].imshow(-sublattice_a.T, cmap='bwr',
                  vmax=1, vmin=-1,
                  alpha=sublattice_a.T)
    axs[i + 1].imshow(sublattice_b.T, cmap='bwr',
                      vmax=1, vmin=-1,
                      alpha=sublattice_b.T)

In [ ]:
sublattice_a , sublattice_a.shape

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(-sublattice_a.T, cmap='bwr',
           vmax=1, vmin=-1,
           alpha=sublattice_a.T)
plt.imshow(sublattice_b.T, cmap='bwr',
           vmax=1, vmin=-1,
           alpha=sublattice_b.T)
plt.imshow(lattice2.T, cmap='PiYG',
           vmax=1, vmin=-1,
           alpha=lattice2.T*0.9)

## Moire Code

In [ ]:
def normalize8(I):
  mn = I.min()
  mx = I.max()

  mx -= mn

  I = ((I - mn)/mx) * 255
  return I.astype(np.uint8)

def create_moire_pattern(width, height, frequency=5, angle=30, intensity=128):
    """
    Creates a moiré pattern with specified frequency, angle, and intensity.

    :param width: Width of the pattern.
    :param height: Height of the pattern.
    :param frequency: Frequency of the sine waves.
    :param angle: Angle of the pattern in degrees.
    :param intensity: Intensity of the pattern (0-255).
    :return: Moiré pattern as a numpy array.
    """
    x = np.linspace(0, width, width)
    y = np.linspace(0, height, height)
    X, Y = np.meshgrid(x, y)

    # Convert angle to radians
    theta = np.deg2rad(angle)

    # Generate sinusoidal pattern
    pattern = np.sin(2 * np.pi * frequency * (np.cos(theta) * X + np.sin(theta) * Y))

    # Normalize the pattern to 0-255 range
    pattern_normalized = ((pattern + 1) / 2 * intensity).astype(np.uint8)

    return pattern_normalized


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the scene image
scene_image = cv2.imread(image_path)

# Ensure the moiré pattern is in a numpy array with the correct dimensions
moire_pattern = create_moire_pattern(500 , 500)  #create_moire_pattern(width, height, frequency=5, angle=30, intensity=128)

# Resize the moiré pattern to match the scene image dimensions
scene_height, scene_width = scene_image.shape[:2]
moire_pattern_resized = cv2.resize(moire_pattern, (scene_width, scene_height))

# Create an alpha channel for the moiré pattern, assuming a constant alpha value (e.g., 0.5)
alpha_value = 0.6
alpha_channel = np.full((scene_height, scene_width), alpha_value, dtype=np.float32)

# Convert the resized moiré pattern to a 3-channel image
moire_pattern_resized_3ch = cv2.cvtColor(moire_pattern_resized, cv2.COLOR_GRAY2BGR)

# Blend the images using the alpha channel
for c in range(0, 3):
    scene_image[:, :, c] = (alpha_channel * moire_pattern_resized_3ch[:, :, c] +
                            (1 - alpha_channel) * scene_image[:, :, c])

# Display the resulting image
cv2_imshow(scene_image)

# Save the resulting image
cv2.imwrite('overlay_result.jpg', scene_image)


## Adding Glare (after Moire)

In [ ]:
# Load the image
ori_img = cv2.imread(image_path)
image1 = cv2.imread('overlay_result.jpg')

# Apply brightness and contrast adjustment
brightness = 60  # Increase brightness
contrast = 30    # Increase contrast
adjusted_image1 = apply_brightness_contrast(image1, brightness, contrast)

# Add natural glare effect at a random location
glare_intensity = 0.3
glare_size = 0.5
adjusted_image1_with_glare = add_natural_glare(adjusted_image1, glare_intensity, glare_size)

# Show the images
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB))

plt.subplot(1, 2, 2)
plt.title('Adjusted Image 1 with Natural Glare')
plt.imshow(cv2.cvtColor(adjusted_image1_with_glare, cv2.COLOR_BGR2RGB))

plt.show()

# Save the adjusted image
cv2.imwrite('adjusted_2025_with_natural_glare.jpg', adjusted_image1_with_glare)

-------------------------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------

# BBAug

In [ ]:
!pip install bbaug

In [ ]:
# Import the packages we need
import inspect
import glob
import os

from imgaug import augmenters as iaa
import imageio
from imgaug.augmentables.bbs import (
    BoundingBox,
    BoundingBoxesOnImage,
)
import numpy as np
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
from bbaug.policies import policies
from bbaug.augmentations import augmentations
from bbaug.policies import list_policies

## BBAug Policies

In [ ]:
print(list_policies())

In [ ]:
policies.policies_v0()

In [ ]:
policies.policies_v1()

In [ ]:
policies.policies_v2()

In [ ]:
policies.policies_v3()

## Dictionary of the augmentation name to the method reference

In [ ]:
from bbaug.augmentations import NAME_TO_AUGMENTATION
NAME_TO_AUGMENTATION # Shows the dictionary of the augmentation name to the method reference

## Customs Policies

In [ ]:
#Customize the Policies as you need following the dictionary showing above

policy = [
  [policies.POLICY_TUPLE('Brightness', 0.5, 3),
  policies.POLICY_TUPLE('Cutout', 0.6, 6),
  policies.POLICY_TUPLE('Contrast', 0.7, 10),
  policies.POLICY_TUPLE('Solarize_Add', 0.4, 3),
  policies.POLICY_TUPLE('Sharpness', 0.6, 6),
  policies.POLICY_TUPLE('Rotate', 0.6, 1),],
]

In [ ]:
import imgaug as ia

img_num = 5551
image_path = f'{PATH}/train/train/images/{img_num}.jpg'
annotation_path = f'{PATH}/train/train/annotations/{img_num}.txt'

im = imageio.imread(image_path)

img_shape = im.shape
bbs = [bbox]
img_aug, bbs_aug = policy_container.apply_augmentation(random_policy, im, bbs, [0])

bbs_aug

# visualize_yolo_annotations(image_path, annotation_path, class_labels)
bbs_aug = BoundingBoxesOnImage([BoundingBox(*box) for box in bbs_aug[:,1:]], img_aug.shape) # Need to convert bounding boxes
ia.imshow(bbs_aug.draw_on_image(img_aug, size=2))

In [ ]:
policy_container = policies.PolicyContainer(
    policy,
    name_to_augmentation=NAME_TO_AUGMENTATION
)
random_policy = policy_container.select_random_policy()
random_policy

## Path for image and annotations

In [ ]:
img_num = 5551

image_path = f'{PATH}/train/train/images/{img_num}.jpg'
annotation_path = f'{PATH}/train/train/annotations/{img_num}.txt'
# visualize_yolo_annotations(image_path, annotation_path, class_labels)

## read image using 'imageio'

In [ ]:
import imageio
from imgaug.augmentables.bbs import (
    BoundingBox,
    BoundingBoxesOnImage,
)

im = imageio.imread('adjusted_2025_with_natural_glare.jpg')

print(f'image shape: {im.shape}\n\n')
im

In [ ]:
with open(annotation_path, 'r') as file:
  annotations = file.readlines()
print(f'annotations: {annotations}')

for annotation in annotations:
  parts = annotation.strip().split()
  class_id = int(parts[0])
  bbox = list(map(float, parts[1:]))
  bbox = denormalize_bbox(im.shape, bbox)

print(f'bbox: {bbox}')

#Apply augment (looping)

In [ ]:
policy_container = policies.PolicyContainer(
    policy,
    name_to_augmentation=NAME_TO_AUGMENTATION
)
random_policy = policy_container.select_random_policy()
random_policy

In [ ]:
img_shape = im.shape
bbs = [bbox]
img_aug, bbs_aug = policy_container.apply_augmentation(random_policy, im, bbs, [0])

bbs_aug

In [ ]:
policy = [[
  # policies.POLICY_TUPLE('Brightness', 0.7, 6),
  # policies.POLICY_TUPLE('Cutout', 0.6, 6),
  # policies.POLICY_TUPLE('Contrast', 0.7, 3),
  # policies.POLICY_TUPLE('Solarize_Add', 0.6, 3),
  # policies.POLICY_TUPLE('Solarize', 0.6, 10),
  # policies.POLICY_TUPLE('Sharpness', 0.6, 6),
  policies.POLICY_TUPLE('Rotate', 0.5, 1),
   ],
]

policy_container = policies.PolicyContainer(
    policy,
    name_to_augmentation=NAME_TO_AUGMENTATION
)
random_policy = policy_container.select_random_policy()
random_policy

In [ ]:
import imgaug as ia

for i in range(20):
  img_aug, bbs_aug = policy_container.apply_augmentation(random_policy, im, bbs, [0])

  bbs_aug = BoundingBoxesOnImage([BoundingBox(*box) for box in bbs_aug[:,1:]], img_aug.shape) # Need to convert bounding boxes
  ia.imshow(bbs_aug.draw_on_image(img_aug, size=2))

---